
<h1><center> <font color='black'>  Network Science (LTAT.02.011) - Homework - 05  (Total points - 10)</font></center></h1>
<h2><center> <font color='black'> Introduction to Network Analysis</font></center></h3>
<h2><center> <font color='black'> University of Tartu</font></center></h3>


# Homework Instructions


- Please provide the names and student IDs of the team-members (Maximum 2 person) in the field "Team mates" below. If you are not working in a team please insert only your name and student ID. 

- ONLY one of the teammates should submit the homework. We will grade the homework and the marks and feedback is applied for both the team members. So please communicate with your team member about marks and feedback if you are submit the homework.


- The accepted submission format is .ipynb file. Please upload the file on moodle.

- The submission will automatically close on **<font color='red'>17th May 2023 at 23:59 pm</font>**, so please make sure you have enough time to submit the homework. 

- You do not necessarily need to work on Colab. Especially as the size and the complexity of datasets will increase through the course, you can install jupyter notebooks locally and work from there.

- If you do not understand what a question is asking for, please ask in Moodle. 

- Please Click on "**Restart and Run All**" before uploading the '.ipynb' file.

- Plagiarism is <font color='red'>**PROHIBITED**</font>. Any form of plagiarism will be dealt according to the university policy (https://www.ut.ee/en/current-students/academic-fraud).

- **Note: The tasks which involves writing your thoughts should not contain more than 3-4 sentences. It should be clear and concise.**


**<h2><font color='red'>Student info:</font></h2>**


<font color='red'>Full name: </font> Samir Musali



In [1]:
!pip install ndlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.7 MB/s eta 0:00:00
  Created wheel for python-igraph: filename=python_igraph-0.10.4-py3-none-any.whl size=9073 sha256=0c69398edffbd39f5a00feef6f69cf062457b056e05f17c707c41c33fb7054e1
  Stored in directory: /root/.cache/pip/wheels/49/27/15/dcf90953e3e3322e6f3e447514b20cf39b53c6518cb4a7bace
Successfully built python-igraph


In [2]:
%matplotlib inline

In [3]:
import itertools

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

no display found. Using non-interactive Agg backend


# 1. SIR: Susceptible-Infected-Recovered (10 points)

Each individual has β contacts with randomly chosen others individuals per unit time.

Each infected individual has μ probability of becoming immune after being infected.

## 1.1 Network creation (3 points)

In this exercise we will create three different networks:
1. Erdos-Renyi random network (1000 nodes connected with probability 0.1)
2. Watts–Strogatz small-world network (1000 nodes connected with k=3 and probability 0.1)
3. Barabási–Albert preferential attachment network (1000 nodes connected with degree 3)

Hint: Refer lab sessions

In [4]:
# Erdos-Renyi random network
er = nx.erdos_renyi_graph(n=1000, p=0.1)

# Watts–Strogatz small-world network
ws = nx.watts_strogatz_graph(n=1000, k=3, p=0.1)

# Barabási–Albert preferential attachment network
ba = nx.barabasi_albert_graph(n=1000, m=3)

## 1.2 Plot susceptible at time stamp 100 with infection and recovery rates between 0 and 1 (7 points) 

In [5]:
def SIR_model(net, infection_rate, recovery_rate):
  model = ep.SIRModel(net)
  model.available_statuses
  cfg = mc.Configuration()
  cfg.add_model_parameter('beta', infection_rate) # infection rate
  cfg.add_model_parameter('gamma', recovery_rate) # recovery rate
  cfg.add_model_parameter("percentage_infected", 0.01)
  model.set_initial_status(cfg)

  iterations = model.iteration_bunch(100, node_status=True)
  trends = model.build_trends(iterations)

  viz = DiffusionTrend(model, trends)
  viz.plot()
  return trends

In [6]:
trends = SIR_model(er, 0.001, 0.01)

### 1.2.1 Return number of susceptible nodes at timestamp 100 (2.5 points)
**<font color='red'>Note: This simulation will take around one hour.</font>**

In [7]:
def SIR_model_S_inf(net, infection_rate, recovery_rate):
  model = ep.SIRModel(net)
  model.available_statuses
  cfg = mc.Configuration()
  cfg.add_model_parameter('beta', infection_rate) # infection rate
  cfg.add_model_parameter('gamma', recovery_rate) # recovery rate
  cfg.add_model_parameter("percentage_infected", 0.01)
  model.set_initial_status(cfg)

  iterations = model.iteration_bunch(100, node_status=True)
  trends = model.build_trends(iterations)
  return trends[0]['trends']['node_count'][0][99]

In [8]:
a = list(np.arange(0, 1.01, 0.02))
df = pd.DataFrame(list(itertools.product(a,a)), columns=['infection_rate', 'recovery_rate'])
df['er'] = 0
df['ws'] = 0
df['ba'] = 0
df.head()

,infection_rate,recovery_rate,er,ws,ba
0,0.0,0.00,0,0,0
1,0.0,0.02,0,0,0
2,0.0,0.04,0,0,0
3,0.0,0.06,0,0,0
4,0.0,0.08,0,0,0


In [9]:
for i in range(len(df)):
  df.loc[i,'er'] = SIR_model_S_inf(er, df.loc[i,'infection_rate'],df.loc[i,'recovery_rate'])
  df.loc[i,'ws'] = SIR_model_S_inf(ws, df.loc[i,'infection_rate'],df.loc[i,'recovery_rate'])
  df.loc[i,'ba'] = SIR_model_S_inf(ba, df.loc[i,'infection_rate'],df.loc[i,'recovery_rate'])

df.head()

,infection_rate,recovery_rate,er,ws,ba
0,0.0,0.00,990,990,990
1,0.0,0.02,990,990,990
2,0.0,0.04,990,990,990
3,0.0,0.06,990,990,990
4,0.0,0.08,990,990,990


### 1.2.2 Plot number of susceptible nodes at timestamp 100 for Erdos-Renyi random network and explain it (1.5 points) 

In [10]:
fig = go.Figure(data=go.Heatmap(
          x = df['infection_rate'], # infection rates
          y = df['recovery_rate'], # recovery rates
          z = df['er'], # susceptible nodes
          type = 'heatmap',
    colorbar=dict(title='Susceptible At T=100')))

fig.update_layout(
    xaxis_title="Infection rate",
    yaxis_title="Recovery rate",
    font_size = 18
)


fig.show()

Please, think about the possible reason why you observe this behaviour. 

**<font color='red'>Your answer:</font>** In the code above:
* df['infection_rate'] is used to provide the infection rates.
* df['recovery_rate'] is used to give the recovery rates.
* df['er'] provides the number of susceptible nodes at timestamp 100 for the Erdos-Renyi network.

This visualization can help us understand how infection and recovery rates influence the network's final number of susceptible individuals. This plot will show a heatmap where the x-axis represents the infection rates, the y-axis represents the recovery rates, and the colour intensity represents the number of susceptible individuals at the end of the simulation (T=100). Darker regions represent combinations of infection and recovery rates that result in more individuals remaining susceptible (not becoming infected), while lighter regions represent combinations that result in fewer sensitive individuals.

The behaviour we're observing is due to the nature of the SIR model and the effect of varying infection and recovery rates on the spread of disease.
1. **Infection Rate (β)**: The infection rate represents how transmissible the disease is. A higher infection rate means the disease spreads more quickly. When the infection rate is very low (e.g., close to 0), the disease does not spread effectively, so most of the population remains susceptible (thus, the yellow region in the plot). The disease spreads more effectively as the infection rate increases, so fewer individuals remain susceptible (blue regions).
2. **Recovery Rate (γ)**: The recovery rate represents how quickly infected individuals recover and gain immunity to the disease. A higher recovery rate means infected individuals recover more quickly, reducing the pool of individuals that can spread the disease. When the recovery rate is low, infected individuals remain infectious for longer, allowing the disease to spread more widely and reducing the number of susceptible individuals.

The region of the plot where the recovery rate is high and the infection rate is low is yellow, indicating many susceptible individuals. This is expected because the disease does not spread effectively (low infection rate), and infected individuals recover quickly (high recovery rate), reducing the opportunity for the disease to spread.

The disease can spread (though not very effectively due to the relatively low infection rate), but infected individuals recover quickly. The region of the plot with a recovery rate of 1 and an infection rate of 0.01 that is a bit yellow (around 400 susceptible individuals) represents a balance between disease transmission and recovery. This combination results in a moderate number of susceptible individuals.

Most of the plot is blue because, for most infection and recovery rate combinations, the disease spreads effectively enough to infect a significant portion of the population, leaving few susceptible individuals. This is particularly true for high infection rates and low recovery rates, where the disease can spread rapidly, and infected individuals remain infectious for longer.

### 1.2.3 Plot number of susceptible nodes at timestamp 100 for Watts–Strogatz small-world network and explain it (1.5 points) 

In [11]:
fig = go.Figure(data=go.Heatmap(
          x = df['infection_rate'], # infection rates
          y = df['recovery_rate'], # recovery rates
          z = df['ws'], # susceptible nodes for Watts-Strogatz
          type = 'heatmap',
    colorbar=dict(title='Susceptible At T=100')))

fig.update_layout(
    xaxis_title="Infection rate",
    yaxis_title="Recovery rate",
    font_size = 18
)


fig.show()

Please, think about the possible reason why you observe this behaviour. 

**<font color='red'>Your answer:</font>** Watts – a high clustering coefficient and a small average shortest path length characterize Strogatz small-world networks. These characteristics can affect how diseases spread in these networks.
1. High Clustering Coefficient: In a Watts-Strogatz network, nodes are likely to be connected to their neighbours' neighbours. This means that if a node becomes infected, its neighbours (and, by extension, their neighbours) are at risk. This can lead to rapid local spread of the disease, especially when the infection rate is high.
2. Small Average Shortest Path Length: This property allows the disease to jump between clusters in the network. Even if the infection rate isn't very high, the disease can still reach a large portion of the network due to its small-world properties.

Consequently, the plot for a Watts-Strogatz network might show fewer susceptible individuals (more blue regions) at a lower infection rate than the Erdos-Renyi network because of the high clustering and small average shortest path length. However, the overall trend will be the same: fewer individuals will remain susceptible as the infection rate increases or the recovery rate decreases. 

As with the previous plot, regions with high recovery rates and low infection rates will have more susceptible individuals (more yellow regions).

The differences between the Erdos-Renyi and Watts-Strogatz plots will depend on the specific parameters of the networks (like the number of nodes, the average degree, the re-wiring probability for the Watts-Strogatz network, etc.).

The behaviour can be understood by considering the nature of the infection and recovery rates in the context of the SIR (Susceptible, Infected, Recovered) model.
1. Infection Rate (ir): When the infection rate is high, a higher percentage of the susceptible population will become infected over time, reducing the number of susceptible individuals. The fact that the susceptibility score decreases as the infection rate increases aligns with this understanding. It means fewer individuals remain susceptible as the infection spreads more efficiently (i.e., higher ir).
2. Recovery Rate (rr): When the recovery rate is high, infected individuals become immune (recovered) faster. This could initially result in more susceptibles (as fewer people are infectious at any given time). However, remember that recovered individuals are permanently immune in the SIR model and do not return to the susceptible pool. Therefore, a high recovery rate doesn't increase the number of susceptibles but decreases the number of infectious individuals more quickly. That's why (ir: 0, rr: 1) results in a high number of susceptibles--because the disease is not spreading (ir: 0), and therefore everyone remains in the susceptible pool.
3. Interaction between ir and rr: When both the infection rate and the recovery rate are high (ir: 1, rr: 1), the disease spreads rapidly, but individuals also recover quickly. However, recovered individuals are assumed to be permanently immune in the SIR model, so they do not return to the susceptible pool. Therefore, the susceptible count still drops to 0.
4. Lower left corner (ir: 0, rr: 0): This is a case with no infection spreading and no recovery. It's a static scenario where everyone stays in the state they started in. Given that we start with 99% of the population susceptible, it makes sense to end with many susceptibles in this scenario. However, when we introduce infection, the number of susceptibles decreases.

This kind of analysis is at the heart of understanding epidemiological models like the SIR model and allows us to predict and understand the spread of diseases in different scenarios.

### 1.2.4 Plot number of susceptible nodes at timestamp 100 for Barabási–Albert preferential attachment network and explain it (1.5 points) 

In [12]:
fig = go.Figure(data=go.Heatmap(
          x = df['infection_rate'], # infection rates
          y = df['recovery_rate'], # recovery rates
          z = df['ba'], # susceptible nodes for Watts-Strogatz
          type = 'heatmap',
    colorbar=dict(title='Susceptible At T=100')))

fig.update_layout(
    xaxis_title="Infection rate",
    yaxis_title="Recovery rate",
    font_size = 18
)


fig.show()

Please, think about the possible reason why you observe this behaviour. 

**<font color='red'>Your answer:</font>** This heatmap illustrates the relationship between infection rate (ir), recovery rate (rr), and the number of susceptible individuals in a Barabási–Albert preferential attachment network. The Barabási–Albert model is a model of scale-free networks, where some nodes are highly connected (hubs), while others have few connections. This characteristic can significantly affect the spread of an infection in the network.

Let's interpret the heatmap and the observed behaviour:
1. **Low Infection Rate (ir near 0)**: The heatmap shows maximum susceptibility (the bright yellow region) when the infection rate is near 0. This makes sense intuitively: fewer individuals are infected when the infection rate is low, and therefore the majority remains susceptible. This is true regardless of the recovery rate, as seen by the high susceptibility extending vertically along the rr axis from rr=0 to rr=1 at low ir.
2. **Increasing Infection Rate (ir increasing from 0 to 0.1)**: As the infection rate increases, the susceptibility remains high for low recovery rates. However, you can see a sharp decrease in susceptibility as ir increases, particularly when rr is low. This is because as the infection rate increases, more individuals become infected and thus leaving the susceptible pool. At low recovery rates, these individuals remain infected longer; thus, the number of susceptible individuals decreases.
3. **High Infection Rate (ir above 0.1)**: Once the infection rate surpasses a certain threshold (around 0.1), the susceptibility drops significantly across all recovery rates, rapidly transitioning to a low susceptibility state. This suggests that once the infection rate is high enough, the infection spreads rapidly throughout the network, quickly depleting the pool of susceptible individuals. This is the steep "cliff" you see in the heatmap.
4. **Influence of Recovery Rate (rr)**: For a given infection rate, increasing the recovery rate has less of an effect on the number of susceptible individuals than increasing the infection rate. This might seem counterintuitive, as one might expect a high recovery rate to keep the number of infectious individuals low and thus reduce the spread of the infection. However, remember that in the SIR model, "recovered" individuals are considered immune and do not return to the susceptible pool. Thus, a high recovery rate does not replenish the susceptible population.
5. **The Role of Preferential Attachment**: The Barabási–Albert network model, characterized by its power-law degree distribution, could contribute to the sharp transition observed in the heatmap. In such a network, a few highly connected nodes (hubs) can quickly spread the infection across the entire network when the infection rate is high. This is consistent with the rapid decrease in susceptible individuals observed as ir increases. Conversely, when the infection rate is low, the disease might not spread effectively, even in the presence of these hubs, resulting in a high number of susceptibles.

In summary, this heatmap provides a rich visualization of how the interplay between infection rate, recovery rate, and the structure of the network (in this case, a Barabási–Albert preferential attachment network) can influence the spread of infection and the resulting number of susceptible individuals at a certain point in time.

## How long did it take you to solve the homework?

* Please answer as precisely as you can. It does not affect your points or grade in any way. It is okay, if it took 0.5 hours or 24 hours. The collected information will be used to improve future homeworks. Please change X in the next cell into your estimate.

<font color='red'> **Answer:** </font> 2 hours




## What is the level of difficulty for this homework?
you can put only number between $0:10$ ($0:$ easy, $10:$ difficult)

<font color='red'> **Answer:** </font> 2